In [2]:
!pip install langchain
!pip install pinecone-client==2.2.4
!pip install openai
!pip install tiktoken
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 6.6 MB/s eta 0:00:00


In [3]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36


In [4]:
!pip install langchain_huggingface

In [5]:
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.vectorstores import Pinecone
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline


In [6]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [7]:
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline
loader = TextLoader("/content/TheStarryNight.txt")

data=loader.load()

In [8]:
data

[Document(metadata={'source': '/content/TheStarryNight.txt'}, page_content='The Starry Night is an oil-on-canvas painting by the Dutch Post-Impressionist painter Vincent van Gogh, painted in June 1889. It depicts the view from the east-facing window of his asylum room at Saint-Rémy-de-Provence, just before sunrise, with the addition of an imaginary village.[1][2][3] It has been in the permanent collection of the Museum of Modern Art in New York City since 1941,[4] acquired through the Lillie P. Bliss Bequest. Widely regarded as Van Gogh\'s magnum opus,[5] The Starry Night is one of the most recognizable paintings in Western art.[6][7]\n\nThe asylum\n\nThe Monastery of Saint-Paul de Mausole\nIn the aftermath of the 23 December 1888 breakdown that resulted in the self-mutilation of his left ear,[8][9] Van Gogh voluntarily admitted himself to the Saint-Paul-de-Mausole lunatic asylum on 8 May 1889.[10][11] Housed in a former monastery, Saint-Paul-de-Mausole catered to the wealthy and was l

Splitting into chunks


In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
text_chunks = text_splitter.split_documents(data)

In [10]:
text_chunks

[Document(metadata={'source': '/content/TheStarryNight.txt'}, page_content="The Starry Night is an oil-on-canvas painting by the Dutch Post-Impressionist painter Vincent van Gogh, painted in June 1889. It depicts the view from the east-facing window of his asylum room at Saint-Rémy-de-Provence, just before sunrise, with the addition of an imaginary village.[1][2][3] It has been in the permanent collection of the Museum of Modern Art in New York City since 1941,[4] acquired through the Lillie P. Bliss Bequest. Widely regarded as Van Gogh's magnum opus,[5] The Starry"),
 Document(metadata={'source': '/content/TheStarryNight.txt'}, page_content='opus,[5] The Starry Night is one of the most recognizable paintings in Western art.[6][7]'),
 Document(metadata={'source': '/content/TheStarryNight.txt'}, page_content='The asylum'),
 Document(metadata={'source': '/content/TheStarryNight.txt'}, page_content='The Monastery of Saint-Paul de Mausole'),
 Document(metadata={'source': '/content/TheStarr

In [11]:
len(text_chunks)

73

In [28]:
from langchain_community.embeddings import HuggingFaceEmbeddings

persist_directory = 'db'

model_name = "sentence-transformers/clip-ViT-B-32"
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    encode_kwargs=encode_kwargs
)

In [29]:
results = hf.embed_query("how are doin")

In [30]:
results

[0.3409597873687744,
 0.21483787894248962,
 -0.030722960829734802,
 -0.0619724839925766,
 0.10276500135660172,
 -0.21069060266017914,
 0.07750460505485535,
 -1.3690980672836304,
 -0.09114101529121399,
 0.21402761340141296,
 -0.2427237629890442,
 -0.0930897518992424,
 -0.35340815782546997,
 -0.19867682456970215,
 0.1030607745051384,
 -0.18310348689556122,
 0.2036297470331192,
 0.1332073211669922,
 -0.020880650728940964,
 -0.034711409360170364,
 0.2759154438972473,
 -0.2729759216308594,
 0.03853745758533478,
 0.07528211921453476,
 -0.2422196865081787,
 -0.04648345708847046,
 -0.03610045090317726,
 -0.012685954570770264,
 -0.30071645975112915,
 -0.1949891895055771,
 -0.011187080293893814,
 0.10705528408288956,
 0.08053746074438095,
 -0.21719714999198914,
 0.17192211747169495,
 -0.18304991722106934,
 -0.02652173861861229,
 0.07203216850757599,
 0.15846115350723267,
 0.02890297770500183,
 0.12497016787528992,
 -0.0029912516474723816,
 -0.08141253888607025,
 0.10500787198543549,
 0.076576888

In [25]:
len(results)

512

In [22]:
with open('/content/PINECONEAPI-KEY.txt', 'r') as file:
    pinecone_key = file.read()

In [34]:
!pip install pinecone
!apt-get install pinecone

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package pinecone


In [46]:
import pinecone
import os

# Initialize Pinecone connection
pinecone.init(api_key=pinecone_key, environment="us-east1-gcp")
 # Ensure "us-east1-gcp" is the correct environment for your starter plan

# Connect to the index
index = pinecone.Index("quickstart")

In [41]:
index_name ='test'

create Embedding for each of the Text chunk

In [42]:
docsearch = Pinecone.from_texts([t.page_content for t in text_chunks], hf, index_name=index_name)

ValueError: No active indexes found in your Pinecone project, are you sure you're using the right Pinecone API key and Environment? Please double check your Pinecone dashboard.

In [39]:
# ... (other imports and code) ...

# Initialize Pinecone connection
pinecone.init(api_key=pinecone_key)

# Create the index if it doesn't exist
if index_name not in pinecone.list_indexes():
    pinecone.create_index(name=index_name, dimension=len(results))

# Now you can create the docsearch object:
docsearch = Pinecone.from_texts([t.page_content for t in text_chunks], hf, index_name=index_name)

ApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-04', 'X-Cloud-Trace-Context': 'cf00ec08379c6d1e60c1ffa9a01ed4a0', 'Date': 'Thu, 21 Nov 2024 15:49:06 GMT', 'Server': 'Google Frontend', 'Content-Length': '136', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: Request failed. You've reach the max pod-based indexes allowed in project Default (0). To add more pod-based indexes, upgrade your plan.


In [43]:
# Initialize Pinecone connection (you already did this)
pinecone.init(api_key=pinecone_key)

# Create the index if it doesn't exist using METADATA_CONFIG for serverless
if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        name=index_name,
        dimension=len(results),  # Assume results is from your hf.embed_query
        metadata_config={"pod_type": "p1"}  # Specify serverless pod type
    )

# Now you can create the docsearch object:
docsearch = Pinecone.from_texts([t.page_content for t in text_chunks], hf, index_name=index_name)

ApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-04', 'X-Cloud-Trace-Context': '4d6fb7c3116acbb5de3ed1f0b2e5ed68', 'Date': 'Thu, 21 Nov 2024 15:51:22 GMT', 'Server': 'Google Frontend', 'Content-Length': '136', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: Request failed. You've reach the max pod-based indexes allowed in project Default (0). To add more pod-based indexes, upgrade your plan.


In [44]:
import pinecone

# Initialize Pinecone connection (you already did this)
pinecone.init(api_key=pinecone_key)

# Create the index if it doesn't exist using METADATA_CONFIG for serverless
if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        name=index_name,
        dimension=len(results),  # Assume results is from your hf.embed_query
        metric="cosine",
        pod_type="s1"
    )

# Now you can create the docsearch object:
docsearch = Pinecone.from_texts([t.page_content for t in text_chunks], hf, index_name=index_name)

ApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-04', 'X-Cloud-Trace-Context': 'f2314616a55dc148ac32b3d3927c2eb1', 'Date': 'Thu, 21 Nov 2024 15:51:30 GMT', 'Server': 'Google Frontend', 'Content-Length': '136', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: Request failed. You've reach the max pod-based indexes allowed in project Default (0). To add more pod-based indexes, upgrade your plan.


In [47]:
pinecone.list_indexes()

[]

In [52]:
import pinecone
pinecone.init(api_key=pinecone_key, environment="us-east1-gcp")
client = pinecone.Client()
index_list = client.list_indexes().names()
print(index_list)

AttributeError: module 'pinecone' has no attribute 'Client'

In [53]:
!pip install pinecone-client --upgrade